In [ ]:
'''
Notebook for running damped SHM experiment
'''
import gpflow
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from scipy.integrate import solve_ivp, odeint
from gpflow.utilities import print_summary, positive, to_default_float, set_trainable
from invariance_kernels import *
from invariance_functions import *
import os
import matplotlib.pyplot as plt
os.environ["CUDA_VISIBLE_DEVICES"] = '3'

In [ ]:
'''
setup the problem
'''
mean = ZeroMean(2)
time_step = 0.01
training_time = 0.22
testing_time = 1

max_x = 2
max_v = 2
n_train = 3
train_starting_position = np.random.uniform(-max_x, max_x, (n_train))
train_starting_velocity = np.random.uniform(-max_v, max_v, (n_train))

print(train_starting_position)
print(train_starting_velocity)
data = get_damped_SHM_data(time_step, training_time, 1e-8, train_starting_position, train_starting_velocity) #switch

In [ ]:
'''
Fitting the model
'''
mean = ZeroMean(2)
moi, *dump = get_GPR_model(get_MOI(), mean, data, 500) #doesn't need dump
print("moi:"+format(moi.log_marginal_likelihood().numpy(),".2f"))
jitter = 1e-4
invar_density = 40
invar_range = 3
poly_f_d = 2
poly_g_d = 2
known_kernel = get_invariance_kernel(DampedSHMInvariance, invar_range, invar_density, jitter) #switch
known, *dump = get_GPR_model(known_kernel, SHMEpsilonMean(known_kernel),data, 500)
print("known: "+format(known.log_marginal_likelihood().numpy(),".2f"))
learnt_kernel = get_parameterised_invariance_kernel(DampedPolynomialInvariance, invar_range, invar_density, jitter, poly_f_d, poly_g_d) #switch
model, fs, gs, coeff = get_GPR_model(learnt_kernel, PolynomialEpsilonMean(learnt_kernel), data, 3000, stored=True)
print("learnt: "+ format(model.log_marginal_likelihood().numpy(),".2f"))
print(learnt_kernel.f_poly.numpy())
print(learnt_kernel.g_poly.numpy())

known_kernel_latent = get_latent_invariance_kernel(SHMLatentInvariance, invar_range, invar_density, jitter) #switch
known_latent, *dump = get_GPR_model(known_kernel_latent, mean, data, 500)
print("known: "+format(known_latent.log_marginal_likelihood().numpy(),".2f"))
learnt_kernel_latent = get_latent_parameterised_invariance_kernel(PolynomialLatentInvariance, invar_range, invar_density, jitter, poly_f_d, poly_g_d) #switch
model_latent, fs_latent, gs_latent, coeff_latent = get_GPR_model(learnt_kernel_latent, mean, data, 3000, stored=True)
print("learnt: " + format(model_latent.log_marginal_likelihood().numpy(),".2f"))
print(learnt_kernel_latent.f_poly.numpy())
print(learnt_kernel_latent.g_poly.numpy())


In [ ]:
'''
plotting posterior of dynamics
'''
import matplotlib.pyplot as plt
test_range = np.radians(150)
test_density = 40
test_xs = tf.linspace(-test_range,test_range,test_density)
test_vs = tf.linspace(-test_range,test_range,test_density)
test_xx, test_vv = tf.meshgrid(test_xs, test_vs)
test_points = tf.stack([tf.reshape(test_xx,[-1]), tf.reshape(test_vv,[-1])], axis=1)
pred, var = moi.predict_f(test_points) ######switch
X, Y = data
pred_a = pred[:int(pred.shape[0]/2),:]
var_a = var[:int(var.shape[0]/2),:]
pred_v = pred[int(pred.shape[0]/2):,:]
var_v = var[int(var.shape[0]/2):,:]
fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw={"projection": "3d"})
ax1.plot_surface(test_xx, test_vv, tf.reshape(pred_a, test_xx.shape), cmap="viridis",linewidth=0, antialiased=False, alpha=0.8)
#ax1.plot_surface(test_xx, test_vv, tf.reshape(pred_a, test_xx.shape)+1.96*tf.math.sqrt(tf.reshape(var_a, test_xx.shape)), color="grey",linewidth=0, antialiased=False, alpha=0.4)
#ax1.plot_surface(test_xx, test_vv, tf.reshape(pred_a, test_xx.shape)-1.96*tf.math.sqrt(tf.reshape(var_a, test_xx.shape)), color="grey",linewidth=0, antialiased=False, alpha=0.4)
ax1.scatter(X[:,0], X[:,1],Y[:,1,None], color="black", marker="o", s=3) #switch for velocity acceleration
ax1.view_init(10,-65)
ax1.set_xlabel("q")
ax1.set_ylabel("p")
ax1.set_zlabel("a") 
ax1.set_title("RBF GP Posterior of a")

ax2.plot_surface(test_xx, test_vv, tf.reshape(pred_v, test_xx.shape), cmap="viridis",linewidth=0, antialiased=False, alpha=0.8)
#ax2.plot_surface(test_xx, test_vv, tf.reshape(pred_v, test_xx.shape)+1.96*tf.math.sqrt(tf.reshape(var_v, test_xx.shape)), color="grey",linewidth=0, antialiased=False, alpha=0.4)
#ax2.plot_surface(test_xx, test_vv, tf.reshape(pred_v, test_xx.shape)-1.96*tf.math.sqrt(tf.reshape(var_v, test_xx.shape)), color="grey",linewidth=0, antialiased=False, alpha=0.4)
ax2.scatter(X[:,0], X[:,1],Y[:,0,None], color="black", marker="o", s=3) #switch for velocity acceleration
ax2.view_init(10,-25)
ax2.set_xlabel("q")
ax2.set_ylabel("p")
ax2.set_zlabel("v") 
ax2.set_title("RBF GP Posterior of v")
plt.savefig("figures/posterior_damped_shm_rbf.pdf")

In [ ]:
'''
compute and plot posterior of the latent dynamics
'''
test_range = 3
test_density = 40
test_xs = tf.linspace(-test_range,test_range,test_density)
test_vs = tf.linspace(-test_range,test_range,test_density)
test_xx, test_vv = tf.meshgrid(test_xs, test_vs)
test_points = tf.stack([tf.reshape(test_xx,[-1]), tf.reshape(test_vv,[-1])], axis=1)
pred = tf.zeros((test_points.shape[0], 1), dtype=tf.float64)
X, Y = data
n = X.shape[0]
m = test_points.shape[0]
FullLatent = FullSHMLatentInvariance(known_kernel_latent)
X1X2 = FullLatent(X, test_points)
X1X1 = FullLatent(X)
X2X2 = FullLatent(test_points)
A = X1X1[:2*n,:2*n]
B = X1X2[:2*n, 2*m:]
C = tf.transpose(B)
D = X2X2[2*m:,2*m:]
y = tf.reshape(tf.transpose(tf.concat([Y[:,1,None],Y[:,0,None]],1)),(Y.shape[0]*2,1))
mean = tf.tensordot(tf.tensordot(C,tf.linalg.inv(A),1), y,1)
cov = D-tf.tensordot(tf.tensordot(C, tf.linalg.inv(A),1), B,1)
var_prior = tf.linalg.diag_part(D)
var_post = tf.linalg.diag_part(cov)

fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw={"projection": "3d"})

ax1.plot_surface(test_xx, test_vv, tf.reshape(pred, test_xx.shape), cmap="viridis",linewidth=0, antialiased=False, alpha=0.8)
ax1.plot_surface(test_xx, test_vv, tf.reshape(pred, test_xx.shape)+1.96*tf.math.sqrt(tf.reshape(var_prior, test_xx.shape)), color="grey",linewidth=0, antialiased=False, alpha=0.4)
ax1.plot_surface(test_xx, test_vv, tf.reshape(pred, test_xx.shape)-1.96*tf.math.sqrt(tf.reshape(var_prior, test_xx.shape)), color="grey",linewidth=0, antialiased=False, alpha=0.4)
ax1.view_init(10,-15)
ax1.set_xlabel("q")
ax1.set_ylabel("p")
ax1.set_zlabel("z") 
ax1.set_title("Prior of Z")

ax2.plot_surface(test_xx, test_vv, tf.reshape(mean, test_xx.shape), cmap="viridis",linewidth=0, antialiased=False, alpha=0.8)
#ax2.plot_surface(test_xx, test_vv, tf.reshape(pred_a, test_xx.shape)+1.96*tf.math.sqrt(tf.reshape(var_post, test_xx.shape)), color="grey",linewidth=0, antialiased=False, alpha=0.4)
#ax2.plot_surface(test_xx, test_vv, tf.reshape(pred_a, test_xx.shape)-1.96*tf.math.sqrt(tf.reshape(var_post, test_xx.shape)), color="grey",linewidth=0, antialiased=False, alpha=0.4)
ax2.view_init(10,-15)
ax2.set_xlabel("q")
ax2.set_ylabel("p")
ax2.set_zlabel("z") 
ax2.set_title("Posterior of Z")


plt.savefig("figures/latent_damped_shm.pdf")

In [ ]:
'''
evaluate the performance
'''
eva_future_moi = []
eva_future_inv = []
eva_future_inv_p = []
eva_future_inv_l = []
eva_future_inv_lp = []

dynamics = damped_SHM_dynamics

# SHM
def energy(X):
    return 0.5*tf.square(X[:,1])+0.5*tf.square(X[:,0])

for i in range(10):
    print(i)
    test_starting_position =(np.random.uniform(-max_x, max_x))
    test_starting_velocity =(np.random.uniform(-max_v, max_v))


    test_starting = (-0.6695609733583971, 0.3211378752275684)#
#    test_starting = (test_starting_position, test_starting_velocity)
    time_setting = (testing_time, time_step)
    print(test_starting)

    evaluate_moi = evaluate_model_future(moi, test_starting, dynamics,time_setting, energy)
    eva_future_moi.append(evaluate_moi[0])
    print(evaluate_moi[0])

#    fig, ax = plt.subplots()
#    plt.scatter(evaluate_moi[4][0,0],evaluate_moi[4][0,1],color="red")
#    plt.plot(evaluate_moi[4][:,0],evaluate_moi[4][:,1], label="truth", color="black", linewidth=2)
#    plt.plot(evaluate_moi[1][:,0],evaluate_moi[1][:,1], "--", label='RBF', color="red")
#    plt.show()

    evaluate_invariance =evaluate_model_future(known, test_starting, dynamics,time_setting, energy)
    eva_future_inv.append(evaluate_invariance[0])
    print(evaluate_invariance[0])
#    plt.scatter(evaluate_moi[4][0,0],evaluate_moi[4][0,1],color="red")
#    plt.plot(evaluate_moi[4][:,0],evaluate_moi[4][:,1], label="truth", color="black", linewidth=2)
#    plt.plot(evaluate_moi[1][:,0],evaluate_moi[1][:,1], "--", label='RBF', color="red")
#    plt.plot(evaluate_invariance[1][:,0],evaluate_invariance[1][:,1], "--", label="known invariance", color="blue")
#    plt.show()

    evaluate_invariance_p =evaluate_model_future(model, test_starting, dynamics, time_setting, energy)#, (m.kernel.inv_f, m.kernel.inv_g), (lambda x: x, lambda x:np.sin(x)))
    eva_future_inv_p.append(evaluate_invariance_p[0])
    print(evaluate_invariance_p[0])

    evaluate_invariance_l = evaluate_model_future(known_latent, test_starting, dynamics,time_setting, energy)
    eva_future_inv_l.append(evaluate_invariance_l[0])
    print(evaluate_invariance_l[0])

    evaluate_invariance_lp = evaluate_model_future(model_latent, test_starting, dynamics, time_setting, energy)#, (m.kernel.inv_f, m.kernel.inv_g), (lambda x: x, lambda x:np.sin(x)))
    eva_future_inv_lp.append(evaluate_invariance_lp[0])
    print(evaluate_invariance_lp[0])

    fig, ax = plt.subplots()
    plt.scatter(evaluate_moi[4][0,0],evaluate_moi[4][0,1],color="red")
    plt.plot(evaluate_moi[4][:,0],evaluate_moi[4][:,1], label="truth", color="black", linewidth=2)
    plt.plot(evaluate_moi[1][:,0],evaluate_moi[1][:,1], "--", label='RBF', color="red")
    plt.plot(evaluate_invariance[1][:,0],evaluate_invariance[1][:,1], "--", label="known invariance", color="blue")
    plt.plot(evaluate_invariance_p[1][:,0],evaluate_invariance_p[1][:,1], "--", label="learnt invariance", color="green")
    plt.plot(evaluate_invariance_l[1][:,0],evaluate_invariance_l[1][:,1], "--", label="known invariance (latent)", color="purple")
    plt.plot(evaluate_invariance_lp[1][:,0],evaluate_invariance_lp[1][:,1], "--", label="learnt invariance (latent)", color="teal")
    plt.xlabel("q")
    plt.ylabel("p")
    plt.legend()
    plt.show()
    print("MSE & "+ format((np.mean(eva_future_moi)),".5f")+" & "+format(np.mean(eva_future_inv),".5f")+" & "+format((np.mean(eva_future_inv_p)),".5f")+" & "+format((np.mean(eva_future_inv_l)),".5f")+" & "+format((np.mean(eva_future_inv_lp)),".5f")+ ' \\\\')



In [ ]:
'''
metrics
'''
print("Log Marginal Likelihood & " + format((moi.log_marginal_likelihood().numpy()),".0f")+" & "+ format((known.log_marginal_likelihood().numpy()),".0f") + " & "+ format((model.log_marginal_likelihood().numpy()),".0f") +" & "+ format((known_latent.log_marginal_likelihood().numpy()),".0f") + " & "+ format((model_latent.log_marginal_likelihood().numpy()),".0f")+ ' \\\\')
print("MSE & "+ format((np.mean(eva_future_moi)),".5f")+" & "+format(np.mean(eva_future_inv),".5f")+" & "+format((np.mean(eva_future_inv_p)),".5f")+" & "+format((np.mean(eva_future_inv_l)),".5f")+" & "+format((np.mean(eva_future_inv_lp)),".5f")+ ' \\\\')

In [ ]:
'''
plotting learnt polynomial
'''
import matplotlib.pyplot as plt
from numpy.polynomial import polynomial
x = np.linspace(-np.radians(150),np.radians(150),100)
fig, (axs) = plt.subplots(2, 2)
axs[0][0].plot(x, -x, label='truth', color="black")
axs[0][1].plot(x, -(x), color="black", label='truth')
axs[1][0].plot(x, -x, label='truth', color="black")
axs[1][1].plot(x, -(x), color="black", label='truth')
axs[0][0].set_title("f(p)")
axs[0][0].set_xlabel("p")
axs[0][0].set_ylabel("f")
axs[0][1].set_title("g(q)")
axs[0][1].set_xlabel("q")
axs[0][1].set_ylabel("g")
for i, (f, g) in enumerate(zip(fs, gs)):
    if i%4==0 and i>-1:
        multiplier = -learnt_kernel.f_poly.transform(f[1])
        f = polynomial.Polynomial(np.squeeze(learnt_kernel.f_poly.transform(f)))
        g = polynomial.Polynomial(np.squeeze(learnt_kernel.g_poly.transform(g)))
        axs[0][0].plot(x, f(x)/multiplier, "--", label='learnt step: '+str(i*10))
        axs[0][1].plot(x, g(x)/multiplier, "--", label='learnt step: '+ str(i*10))
for i, (f, g) in enumerate(zip(fs_latent, gs_latent)):
    if i%1==0 and i>-1:
        multiplier = -learnt_kernel_latent.f_poly.transform(f[1])
        f = polynomial.Polynomial(np.squeeze(learnt_kernel_latent.f_poly.transform(f)))
        g = polynomial.Polynomial(np.squeeze(learnt_kernel_latent.g_poly.transform(g)))
        axs[1][0].plot(x, f(x)/multiplier, "--", label='learnt step: '+str(i*10))
        axs[1][1].plot(x, g(x)/multiplier, "--", label='learnt step: '+ str(i*10))
axs[0][0].legend(loc=1, prop={"size":7})
axs[0][1].legend(loc=1, prop={"size":7})
axs[1][0].legend(loc=1, prop={"size":7})
axs[1][1].legend(loc=3, prop={"size":7})
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)
plt.savefig("figures/damped_shm_learnt_over_time.pdf")



In [ ]:
'''
plotting a predicted trajectory
'''
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
plt.scatter(evaluate_moi[4][0,0],evaluate_moi[4][0,1],color="red")
plt.plot(evaluate_moi[4][:,0],evaluate_moi[4][:,1], label="truth", color="black", linewidth=2)
plt.plot(evaluate_moi[1][:,0],evaluate_moi[1][:,1], "--", label='RBF', color="red")
plt.plot(evaluate_invariance[1][:,0],evaluate_invariance[1][:,1], "--", label="known invariance", color="blue")
plt.plot(evaluate_invariance_p[1][:,0],evaluate_invariance_p[1][:,1], "--", label="learnt invariance", color="green")
plt.plot(evaluate_invariance_l[1][:,0],evaluate_invariance_l[1][:,1], "--", label="known invariance (latent)", color="purple")
plt.plot(evaluate_invariance_lp[1][:,0],evaluate_invariance_lp[1][:,1], "--", label="learnt invariance (latent)", color="teal")
plt.xlabel("q")
plt.ylabel("p")
plt.legend()
plt.savefig("figures/damped_shm_predicted.pdf")

In [ ]:
'''
plotting the energy along the predicted trajectory
'''
t = np.linspace(0, testing_time, int(testing_time/time_step))
plt.plot(t, tf.squeeze(evaluate_invariance[5])+1, label="truth", color="black")
plt.plot(t, tf.squeeze(evaluate_moi[6])+1, "--", label="RBF", color="red")
plt.plot(t, tf.squeeze(evaluate_invariance[6])+1,"--" ,  label="known invariance", color="blue")
plt.plot(t, tf.squeeze(evaluate_invariance_p[6])+1, "--",label="learnt invariance", color="green")
plt.plot(t, tf.squeeze(evaluate_invariance_l[6])+1,"--" ,  label="known invariance (latent)", color="purple")
plt.plot(t, tf.squeeze(evaluate_invariance_lp[6])+1, "--",label="learnt invariance (latent)", color="teal")
plt.legend()
plt.xlabel("t")
plt.ylabel("E")
plt.savefig("figures/damped_shm_energy.pdf")